# Exercise 5.4.6

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from ISLP import load_data

Default = load_data('Default')


We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the glm() function. Do not forget to set a random seed before beginning your analysis.

a. Determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors

In [ ]:
#6a
# Set random seed
np.random.seed(10)

# Fit logistic regression model
model = smf.glm("default ~ income + balance", data=Default, family=sm.families.Binomial(link=sm.families.links.Logit()))
result = model.fit()

# Print summary
print(result.summary())#some silly plotting issue with se of balance ?
print(result.bse) #0.000005 and 0.0002

b. Write a function, boot.fn(), that takes as input the Default dataset as well as an index of the observations. 

In [ ]:
#6b
def boot_fn(data, index):
    sample = data.iloc[index]
    model = smf.glm("default ~ income + balance", data=sample, family=sm.families.Binomial(link=sm.families.links.Logit()))
    result = model.fit()
    return result.params[1:3]
rng = np.random.default_rng(0)
index=rng.choice(Default.shape[0], Default.shape[0],replace=True)
print(boot_fn(Default,index))

c. Estimate the standard errors of the logistic regression coefficients with bootstrap. (see lab exercises for an example)
d. Comment on the estimated standard errors obtained using the glm() function and using your bootstrap function.

In [ ]:
#6c&d
def boot_SE(func, D, n=None, B=1000, seed=0):
    #Compute bootstrap standard errors for coefficients
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    
    return np.sqrt(second_ / B - (first_ / B)**2)
print(boot_SE(boot_fn,Default,n=Default.shape[0],B=1000))


The estimated standard errors are nearly equal, and are comparable. The bootstrap approach does not rely on many assumptions and should give a good estimation of the real standard error.